# Estimating engine remaining useful life

Author: Luan Glasser

E-mail: luan.glasser@gmail.com

## Information taken from C-MAPSS

>**Datasets**
>
>Data Set: FD001 |
>Train trajectories: 100 | 
>Test trajectories: 100 | 
>Conditions: ONE (Sea Level) | 
>Fault Modes: ONE (HPC Degradation)
>
>Data Set: FD002 | 
>Train trajectories: 260 | 
>Test trajectories: 259 | 
>Conditions: SIX | 
>Fault Modes: ONE (HPC Degradation) 
>
>Data Set: FD003 | 
>Train trajectories: 100 | 
>Test trajectories: 100 | 
>Conditions: ONE (Sea Level) | 
>Fault Modes: TWO (HPC Degradation, Fan Degradation) 
>
>Data Set: FD004 | 
>Train trajectories: 248 | 
>Test trajectories: 249 | 
>Conditions: SIX | 
>Fault Modes: TWO (HPC Degradation, Fan Degradation)  
>
>
>
>**Experimental Scenario**
>
>Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine – i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. The data is contaminated with sensor noise.
>
>The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The objective of the competition is to predict the number of remaining operational cycles before failure in the test set, i.e., the number of operational cycles after the last cycle that the engine will continue to operate. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.
>
>The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:
>
>1)	unit number
>
>2)	time, in cycles
>
>3)	operational setting 1
>
>4)	operational setting 2
>
>5)	operational setting 3
>
>6)	sensor measurement  1
>
>7)	sensor measurement  2
>
>...
>
>26)	sensor measurement  20
>
>
>
>**Reference** 
>
>A. Saxena, K. Goebel, D. Simon, and N. Eklund, “Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation”, in the Proceedings of the Ist International Conference on Prognostics and Health Management (PHM08), Denver CO, Oct 2008.

The sensors features in the datasets correspond to the table bellow.

| Index 	|               Predictor name               	|   Unit  	|
|:-----:	|:------------------------------------------:	|:-------:	|
|   1   	| Total temperature at fan inlet             	|    Ko   	|
|   2   	| Total temperature at LPC outlet            	|    Ko   	|
|   3   	| Total temperature at HPC outlet            	|    Ko   	|
|   4   	| Total temperature at LPT outlet            	|    Ko   	|
|   5   	| Pressure at fan inlet                      	|   psia  	|
|   6   	| Total pressure in bypass-duct              	|   psia  	|
|   7   	| Total pressure HPC outlet                  	|   psia  	|
|   8   	| Physical fan speed                         	|   rpm   	|
|   9   	| Physical core speed                        	|   rpm   	|
|   10  	| Engine pressure ratio                      	|    -    	|
|   11  	| Static pressure at HPC outlet              	|   psia  	|
|   12  	| Ratio of fuel flow to “16”                 	| pps/psi 	|
|   13  	| Corrected fan speed                        	|   rpm   	|
|   14  	| Corrected core speed                       	|   rpm   	|
|   15  	| Bypass ratio  – 16 Burner fuel-air   ratio 	|    –    	|
|   17  	| Bleed Bleed enthalpy                       	|    –    	|
|   18  	| Demanded fan speed                         	|   rpm   	|
|   19  	| Demanded core fan speed                    	|   rpm   	|
|   20  	| HPT coolant bleed                          	|  lbm/s  	|
|   21  	| LPT coolant bleed                          	|  lbm/s  	|

## General libraries

In [122]:
# Import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Loading data

In [123]:
# Load the training data
df_train_fd001 = pd.read_csv('../dataset_nasa/train_FD001.txt', sep = " ", header = None).drop(columns = [26, 27])
df_train_fd002 = pd.read_csv('../dataset_nasa/train_FD002.txt', sep = " ", header = None).drop(columns = [26, 27])
df_train_fd003 = pd.read_csv('../dataset_nasa/train_FD003.txt', sep = " ", header = None).drop(columns = [26, 27])
df_train_fd004 = pd.read_csv('../dataset_nasa/train_FD004.txt', sep = " ", header = None).drop(columns = [26, 27])

# Load the test data
df_test_fd001 = pd.read_csv('../dataset_nasa/test_FD001.txt', sep = " ", header = None).drop(columns = [26, 27])
df_test_fd002 = pd.read_csv('../dataset_nasa/test_FD002.txt', sep = " ", header = None).drop(columns = [26, 27])
df_test_fd003 = pd.read_csv('../dataset_nasa/test_FD003.txt', sep = " ", header = None).drop(columns = [26, 27])
df_test_fd004 = pd.read_csv('../dataset_nasa/test_FD004.txt', sep = " ", header = None).drop(columns = [26, 27])

# Load the RUL data
df_rul_fd001 = pd.read_csv('../dataset_nasa/RUL_FD001.txt', sep = " ", header = None).drop(columns = [1])
df_rul_fd002 = pd.read_csv('../dataset_nasa/RUL_FD002.txt', sep = " ", header = None).drop(columns = [1])
df_rul_fd003 = pd.read_csv('../dataset_nasa/RUL_FD003.txt', sep = " ", header = None).drop(columns = [1])
df_rul_fd004 = pd.read_csv('../dataset_nasa/RUL_FD004.txt', sep = " ", header = None).drop(columns = [1])

## Renaming features

In [124]:
# Define names
names_list = ['unit_no', 'time_cycles', 'op_set_1', 'op_set_2', 'op_set_3', 
        'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 
        'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 
        'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 
        'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 
        'sensor_21']

# Change train features
df_train_fd001.columns = names_list
df_train_fd002.columns = names_list
df_train_fd003.columns = names_list
df_train_fd004.columns = names_list


# Change test features
df_test_fd001.columns = names_list
df_test_fd002.columns = names_list
df_test_fd003.columns = names_list
df_test_fd004.columns = names_list

# Change RUL feature
names_list = ['rul']
df_rul_fd001.columns = names_list
df_rul_fd002.columns = names_list
df_rul_fd003.columns = names_list
df_rul_fd004.columns = names_list

## Data preparation: basic functions

First let's create the functions for generating labels for every line of the training datasets. The labels are for each line the actual operating cycles subtracted from the failure cycles.

### Assigning RUL function

In [125]:
# Group by engine number
def get_max_cycles_per_unit(df):
    df = df.groupby('unit_no', as_index = False).max()[['unit_no', 'time_cycles']]
    df.columns = ['engine_no', 'max']
    return df

# Calculate the RUL for the training dataset
def calculate_rul(df, rul_df):
    df = df.merge(rul_df, how = 'left', left_on = 'unit_no', right_on = 'engine_no')
    df['rul'] = df['max'] - df['time_cycles']
    df =df.drop(columns = ['engine_no', 'max'])
    return df

### Feature selection based on correlation function

The correlation matrix will be computed and the features whose correlations are higher than 0.5 or lower than -0.5 will be selected.

In [126]:
def get_features(df, corr_parameter_low = 0.4, corr_parameter_high = 0.9, show_plot = False):
    
    # Computing correlation matrix 
    correlation_df = df.corr()
    
    if show_plot:
        # Print
        plt.figure(figsize = (12, 12))
        ax = sns.heatmap(
        correlation_df, 
        vmin = -1, vmax = 1, center = 0,
        cmap = sns.diverging_palette(20, 220, n = 10),
        square = True, linewidths = 1.0,
        )
        ax.set_xticklabels(
            ax.get_xticklabels(),
            rotation = 45,
            horizontalalignment = 'right'
        )

    # Select features
    selected_features_list = list(correlation_df[((correlation_df.rul > corr_parameter_low) & (correlation_df.rul < corr_parameter_high)) | \
                                                 ((correlation_df.rul < -corr_parameter_low) & (correlation_df.rul > -corr_parameter_high))].index)
    # Append the rul feature
    selected_features_list.append('rul')
    
    # Print selected features
    print(selected_features_list)
    
    # Filtering the data
    df = df[selected_features_list]
    
    return df

### Scaling the data functions

In [127]:
# Get preprocessing library
from sklearn import preprocessing

def scale_data(df):
    
    # Transform data in X_train and y_train
    X = df.loc[:, df.columns != 'rul'].values
    
    try:
        y = df['rul'].values
    except:
        y = None

    # Scale X_train
    X_scaled = preprocessing.scale(X)
    
    return X, y, X_scaled

In [128]:
from sklearn.preprocessing import Normalizer

def normalize_data(df):
    
    # Transform data in X_train and y_train
    X = df.loc[:, df.columns != 'rul'].values
    
    try:
        y = df['rul'].values
    except:
        y = None

    # Scale X_train
    transformer = Normalizer().fit(X)
    X_scaled = transformer.transform(X)
    
    return X, y, X_scaled

### Linear regression model function

In [129]:
# Import linear regression
from sklearn.linear_model import LinearRegression

# Linear regression function
def run_linear_regression(X, y):
    reg = LinearRegression().fit(X, y)
    print(reg.score(X, y))
    return reg

### Support vector machine classifier model function

In [130]:
# Import SVC
from sklearn.svm import SVC

# Run SVC model
def run_svc(X, y):
    clf = SVC()
    clf.fit(X, y)
    return clf

## Naive modeling

In this section, a naive model will be made. The modeling is naive because for all the failure modes (all the datasets) the modeling will assume that the same approach could be applied. 

### General model

In [131]:
# Run general model
def run_model(X, y, model):
    model.fit(X, y)
    return model

### Lists of datasets to use in general modeling

Bellow the datasets are transformed in lists, so we can apply the general models for all of them at once.

In [132]:
# Saving the data into lists of dataframes
train_list = [df_train_fd001, df_train_fd002, df_train_fd003, df_train_fd004]
test_list = [df_test_fd001, df_test_fd002, df_test_fd003, df_test_fd004]
rul_list = [df_rul_fd001, df_rul_fd002, df_rul_fd003, df_rul_fd004]

### Generalization: linear regression, SVC and SVR preliminar tests

In [133]:
# Importing train, test and split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score

In [134]:
def train_model(train_list, rul_list, model, corr_parameter = 0.5, scaled = True):
    
    # Creating lists to save results
    trained_model_list = []
    i = 1
    # Run the model for each failure mode -- aka each dataset related to a failure mode
    for df_train, df_rul in zip(train_list, rul_list):
        print('MODEL FD00{}'.format(i))
        # Prepare data with one RUL per line
        engine_ruls_df = get_max_cycles_per_unit(df_train)
        df_train = calculate_rul(df_train, engine_ruls_df)
        # Get best features (naive selection, get all features with corr > 0.5 or < -0.5)
        df_train = get_features(df_train, corr_parameter_low = 0.4, corr_parameter_high = 0.9)
        # Scaling the data for the model generation
        X, y, X_scaled = scale_data(df_train)
        # Use scaled data?
        if scaled:
            X = X_scaled
        # Train, test and split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)
        # Modeling
        model = run_model(X_train, y_train, model)
        # Predict
        y_pred = model.predict(X_test)
        # Scores 
        print('The model has got {} predictions in a range of +/- 40 cycles from true value out of {} (ratio = {:.2f}).'.format((np.abs(y_pred - y_test) <= 40).sum(), len(y_pred), (np.abs(y_pred - y_test) <= 40).sum()/len(y_pred)))
        print('The model has got {} predictions in a range of +/- 20 cycles from true value out of {} (ratio = {:.2f}).'.format((np.abs(y_pred - y_test) <= 20).sum(), len(y_pred), (np.abs(y_pred - y_test) <= 20).sum()/len(y_pred)))
        print('The model has got {} predictions in a range of +/- 10 cycles from true value out of {} (ratio = {:.2f}).'.format((np.abs(y_pred - y_test) <= 10).sum(), len(y_pred), (np.abs(y_pred - y_test) <= 10).sum()/len(y_pred)))
        print('The model has got {} predictions in a range of +/- 5 cycles from true value out of {} (ratio = {:.2f}).'.format((np.abs(y_pred - y_test) <= 5).sum(), len(y_pred), (np.abs(y_pred - y_test) <= 5).sum()/len(y_pred)))
        print('\n')
        i += 1
        # Saving results in list
        trained_model_list.append(model)
    
    # Return list of trained models
    return trained_model_list[0], trained_model_list[1], trained_model_list[2], trained_model_list[3]

In [135]:
# Run of linear regression Model with UNscaled data
model = LinearRegression()
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = False)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 1506 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.73).
The model has got 829 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.40).
The model has got 449 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.22).
The model has got 220 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.11).


MODEL FD002
['time_cycles', 'rul']
The model has got 3314 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.62).
The model has got 1772 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.33).
The model has got 879 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.16).
The model has got 469 predictions

In [28]:
# Run of linear regression Model with scaled data
model = LinearRegression()
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = True)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 1506 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.73).
The model has got 829 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.40).
The model has got 449 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.22).
The model has got 220 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.11).


MODEL FD002
['time_cycles', 'rul']
The model has got 3314 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.62).
The model has got 1772 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.33).
The model has got 879 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.16).
The model has got 469 predictions

In [29]:
# Run of SVC model with UNscaled data
model = SVC()
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = False)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 918 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.44).
The model has got 458 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.22).
The model has got 248 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.12).
The model has got 136 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.07).


MODEL FD002
['time_cycles', 'rul']
The model has got 3654 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.68).
The model has got 2166 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.40).
The model has got 1119 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.21).
The model has got 616 predictions

In [30]:
# Run of SVC model with scaled data
model = SVC()
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = True)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 1551 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.75).
The model has got 1070 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.52).
The model has got 696 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.34).
The model has got 421 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.20).


MODEL FD002
['time_cycles', 'rul']
The model has got 3654 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.68).
The model has got 2166 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.40).
The model has got 1119 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.21).
The model has got 616 predictio

In [31]:
from sklearn.svm import SVR
# Run of SVC model with scaled data
model = SVR(kernel = 'rbf', C = 100, gamma = 0.1, epsilon = 0.1)
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = True)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 1633 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.79).
The model has got 1183 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.57).
The model has got 739 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.36).
The model has got 427 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.21).


MODEL FD002
['time_cycles', 'rul']
The model has got 3715 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.69).
The model has got 2188 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.41).
The model has got 1275 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.24).
The model has got 666 predictio

In [136]:
from sklearn.svm import SVR
# Run of SVC model with UNscaled data
model = SVR(kernel = 'rbf', C = 100, gamma = 0.1, epsilon = 0.1)
reg_model_fd001, reg_model_fd002, reg_model_fd003, reg_model_fd004 = train_model(train_list, 
                                                                                 rul_list, 
                                                                                 model = model, 
                                                                                 corr_parameter = 0.5, 
                                                                                 scaled = False)

MODEL FD001
['time_cycles', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21', 'rul']
The model has got 1524 predictions in a range of +/- 40 cycles from true value out of 2064 (ratio = 0.74).
The model has got 972 predictions in a range of +/- 20 cycles from true value out of 2064 (ratio = 0.47).
The model has got 547 predictions in a range of +/- 10 cycles from true value out of 2064 (ratio = 0.27).
The model has got 280 predictions in a range of +/- 5 cycles from true value out of 2064 (ratio = 0.14).


MODEL FD002
['time_cycles', 'rul']
The model has got 3712 predictions in a range of +/- 40 cycles from true value out of 5376 (ratio = 0.69).
The model has got 2186 predictions in a range of +/- 20 cycles from true value out of 5376 (ratio = 0.41).
The model has got 1256 predictions in a range of +/- 10 cycles from true value out of 5376 (ratio = 0.23).
The model has got 649 prediction